Using the `gensim` library's Latent Dirichlet Allocation model to do topic modeling.
Notebook derived from tutorial here: https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

In [1]:
import re
import warnings

import gensim
from   gensim import corpora
from   gensim.models import CoherenceModel
from   gensim.utils import simple_preprocess
import matplotlib.pyplot as plt
from   nltk.corpus import stopwords
from   nltk.stem import WordNetLemmatizer
import numpy as np
import pandas as pd
from   pprint import pprint
import pyLDAvis
import pyLDAvis.gensim

warnings.filterwarnings('ignore', category=DeprecationWarning)

In [2]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [3]:
DATA = '../../data'

In [4]:
!ls ../../data

GoogleNews-vectors-negative300.bin fr_embeddings.pkl
WSJ_02-21.pos                      hmm_vocab.txt
WSJ_24.pos                         newsgroups.json
aclImdb_v1.tar.gz                  nmt
ag_news                            sarcasm.json
bbc-text.csv                       shakespeare.txt
books                              surnames
capitals.txt                       test.words
en-fr.test.txt                     twitter_samples
en-fr.train.txt                    wiki.multi.fr.vec
en_US.twitter.txt                  yelp
en_embeddings.pkl


In [5]:
df = pd.read_json(f'{DATA}/newsgroups.json')
print(df.target_names.unique())
df.head()

['rec.autos' 'comp.sys.mac.hardware' 'comp.graphics' 'sci.space'
 'talk.politics.guns' 'sci.med' 'comp.sys.ibm.pc.hardware'
 'comp.os.ms-windows.misc' 'rec.motorcycles' 'talk.religion.misc'
 'misc.forsale' 'alt.atheism' 'sci.electronics' 'comp.windows.x'
 'rec.sport.hockey' 'rec.sport.baseball' 'soc.religion.christian'
 'talk.politics.mideast' 'talk.politics.misc' 'sci.crypt']


,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,comp.graphics
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space


In [6]:
data = df.content.values.tolist()
# Scrub email addresses
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
# Rm newlines
data = [re.sub('\s+', ' ', sent) for sent in data]
# Rm single quotes
data = [re.sub("\'", '', sent) for sent in data]
pprint(data[:1])

['From: (wheres my thing) Subject: WHAT car is this!? Nntp-Posting-Host: '
 'rac3.wam.umd.edu Organization: University of Maryland, College Park Lines: '
 '15 I was wondering if anyone out there could enlighten me on this car I saw '
 'the other day. It was a 2-door sports car, looked to be from the late 60s/ '
 'early 70s. It was called a Bricklin. The doors were really small. In '
 'addition, the front bumper was separate from the rest of the body. This is '
 'all I know. If anyone can tellme a model name, engine specs, years of '
 'production, where this car is made, history, or whatever info you have on '
 'this funky looking car, please e-mail. Thanks, - IL ---- brought to you by '
 'your neighborhood Lerxst ---- ']


In [7]:
def sent2words(sentences):
    for sent in sentences:
        # deacc=True: remove punctuation
        yield (gensim.utils.simple_preprocess(str(sent), deacc=True))

In [8]:
data_words = list(sent2words(data))
print(data_words[0])

['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'edu', 'organization', 'university', 'of', 'maryland', 'college', 'park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']


In [9]:
# bi-/tri-gram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

print(trigram_mod[bigram_mod[data_words[0]]])

['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp_posting_host', 'rac_wam_umd_edu', 'organization', 'university', 'of', 'maryland_college_park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front_bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']


In [25]:
def remove_stops(texts, stop_words):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words]
            for doc in texts]

In [26]:
def make_bigrams(texts, bigram_mod):
    return [bigram_mod[doc] for doc in texts]

In [27]:
def make_trigrams(texts, bigram_mod, trigram_mod):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

In [28]:
wn = WordNetLemmatizer()

In [43]:
def lemmatize(texts, lemmatizer):
    texts_out = []
    for sent in texts:
        doc = [lemmatizer.lemmatize(word) for word in sent]
        texts_out.append(doc)
    return texts_out

In [30]:
data_words_nostops = remove_stops(data_words, stop_words)

In [33]:
#data_words_nostops[0]

In [35]:
data_words_bigrams = make_bigrams(data_words_nostops, bigram_mod)

In [44]:
#data_words_bigrams[0]

In [45]:
data_lemmatized = lemmatize(data_words_bigrams, wn)

In [47]:
#data_lemmatized[0]